In [1]:
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab
! pip install torch==1.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 194kB 11.7MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [2]:
# from fastai.text.all import *
# import pickle

from fastai.text.all import *

In [3]:
from google.colab import drive
drive.mount('/content/drive')
# ! cd /content/drive/My\ Drive/DVA-Project/data/; unzip huffpost25.zip




Mounted at /content/drive


# Copy data to session

In [4]:
! cp -r "/content/drive/My Drive/DVA-Project/data/huffpost25.zip" .
! unzip huffpost25.zip
! cp "/content/drive/My Drive/DVA-Project/data/outputs/news_lm.pickle" .
! cp "/content/drive/My Drive/DVA-Project/data/outputs/models/acc83.pth" .
! cp "/content/drive/My Drive/DVA-Project/data/all_sampled2.csv" .

Streaming output truncated to the last 5000 lines.
  inflating: huffpost25/train/COLLEGE/train_925.txt  
  inflating: huffpost25/train/COLLEGE/train_931.txt  
  inflating: huffpost25/train/COLLEGE/train_919.txt  
  inflating: huffpost25/train/COLLEGE/train_702.txt  
  inflating: huffpost25/train/COLLEGE/train_716.txt  
  inflating: huffpost25/train/COLLEGE/train_54.txt  
  inflating: huffpost25/train/COLLEGE/train_299.txt  
  inflating: huffpost25/train/COLLEGE/train_40.txt  
  inflating: huffpost25/train/COLLEGE/train_68.txt  
  inflating: huffpost25/train/COLLEGE/train_97.txt  
  inflating: huffpost25/train/COLLEGE/train_528.txt  
  inflating: huffpost25/train/COLLEGE/train_83.txt  
  inflating: huffpost25/train/COLLEGE/train_500.txt  
  inflating: huffpost25/train/COLLEGE/train_266.txt  
  inflating: huffpost25/train/COLLEGE/train_272.txt  
  inflating: huffpost25/train/COLLEGE/train_514.txt  
  inflating: huffpost25/train/COLLEGE/train_448.txt  
  inflating: huffpost25/train/COLLEG

In [5]:
# data_path = Path("/content/drive/My Drive/DVA-Project/data/huffpost25/")
data_path = Path("huffpost25/")
vocab = pickle.load(open("news_lm.pickle", 'rb'))
dls = TextDataLoaders.from_folder(path=data_path, train='train', valid='test', text_vocab = vocab)

learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
learn.load("/content/acc83")
index_to_cat = dls.vocab[1]


In [6]:
import time
import tqdm
csv.field_size_limit(100000000)
csvpath = "all_sampled2.csv"
num_articles = 1e10

articles_list = []
with open(csvpath) as csv_file:
    
    csv_reader = csv.reader(csv_file, delimiter=',')
    t0 = time.time()
    for i, row in enumerate(csv_reader):
        # if i%1000==0:
        #   print(i)
        if i>=num_articles:
            break
        article = ",".join(row[7:-3])
        date = row[2].strip().split(' ')[0].replace('-', '')[1:7]+"01"
        source = row[-1].split("'")[1]
        link = row[-3]
        articles_list.append(article)
        # pred = learn.predict(article)
        # results.append(int(pred[1]))

    t1 = time.time() - t0
    print(t1)

print(len(articles_list))



13.872703552246094
231585


In [14]:
t0 = time.time()
test_dl = dls.test_dl(articles_list)
t1 = time.time() - t0
print(t1)

t0 = time.time()
preds = learn.get_preds(dl=test_dl)
t1 = time.time() - t0
print(t1)


1546.827807188034


2970.5115098953247


In [15]:
import pickle

preds_ = preds[0]
num_articles = len(preds_)
preds_ = np.array(preds_)
results = np.argmax(preds_, axis=1)
# max_probs = np.max(preds, axis = 1)
results_dump = []
for res in results:
  result_dict = {}
  result_dict['pred_idx'] = res
  result_dict['pred_label'] = index_to_cat[res]
  results_dump.append(result_dict)
# print(results_dump)
with open("preds.txt", 'w') as outfile:
  for elem in results_dump:
    outfile.write("{} {}\n".format(elem['pred_idx'], elem['pred_label']))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
! cp preds.txt "/content/drive/My Drive/DVA-Project/data/outputs/"

In [21]:
lines = open("preds.txt").read().split("\n")
print(lines[10000])

7 ENVIRONMENT
